In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

In [3]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [16]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var')(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [18]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
vae_enc=make_encoder(57,32,16,3)
vae_dec=make_decoder(57,32,16,3)
vae_40MHZ=VAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Different_VAE_Models/non_normed_new_beta_{}_v4/'.format(beta),save_format='tf')

Epoch 1/100
18000/18000 [==============================] - 49s 3ms/step - loss: 23252.5475 - reco_loss: 3630.5920 - kl_loss: 3382.8665 - val_loss: 945.9092 - val_reco_loss: 8.8327 - val_kl_loss: 937.0765 - lr: 0.0010
Epoch 2/100
18000/18000 [==============================] - 46s 3ms/step - loss: 18.6809 - reco_loss: 15.6131 - kl_loss: 2.9534 - val_loss: 835.1618 - val_reco_loss: 9.5596 - val_kl_loss: 825.6022 - lr: 0.0010
Epoch 3/100
18000/18000 [==============================] - 46s 3ms/step - loss: 16.6117 - reco_loss: 13.8992 - kl_loss: 2.6592 - val_loss: 773.0566 - val_reco_loss: 7.1584 - val_kl_loss: 765.8982 - lr: 0.0010
Epoch 4/100
18000/18000 [==============================] - 46s 3ms/step - loss: 902409.0312 - reco_loss: 884351.8028 - kl_loss: 13007.3604 - val_loss: 784.4340 - val_reco_loss: 6.7893 - val_kl_loss: 777.6447 - lr: 0.0010
Epoch 5/100
18000/18000 [==============================] - 46s 3ms/step - loss: 15.1461 - reco_loss: 12.5065 - kl_loss: 2.6226 - val_loss: 765.1

In [19]:
a=Data_Train_Flat[0:100]
b=vae_40MHZ.encoder(a)
print(b)

[<tf.Tensor: shape=(100, 3), dtype=float32, numpy=
array([[ 4.31914687e-01, -7.97375023e-01, -1.51638985e+00],
       [-1.65254939e+00, -1.82340443e-01,  1.42557144e+00],
       [-6.27630830e-01, -5.48396647e-01,  8.90338361e-01],
       [-1.69247043e+00, -7.02982962e-01,  7.27045834e-01],
       [-1.67967439e-01, -3.08691561e-01,  8.30240071e-01],
       [-4.05854821e-01,  1.08418977e+00, -9.63941216e-02],
       [ 2.00008392e-01,  9.25589085e-01, -8.74878466e-02],
       [ 1.10806060e+00, -5.09554207e-01, -1.22455978e+00],
       [ 6.52772069e-01,  5.26440144e-02, -1.17008656e-01],
       [ 5.90600610e-01,  6.62524879e-01, -1.17901170e+00],
       [-6.69216275e-01,  1.05173016e+00, -7.16586113e-02],
       [-6.34898067e-01,  8.49854052e-01, -3.81673247e-01],
       [ 2.08062768e-01,  1.09507322e-01,  4.67751801e-01],
       [ 1.45472765e-01,  1.83409870e-01, -2.23564684e-01],
       [-9.93127465e-01,  2.10957587e-01,  2.44213760e-01],
       [ 1.98210907e+00, -4.35363352e-01, -7.5906